In [1]:
import tensorflow as tf
import numpy as np

In [2]:
def pad_and_concat(sequences):  # sequences shape: [batch_size, len, dims...] -> ([batch_size, maxlen, dims...], [len])
    arrays = [np.asarray(seq) for seq in sequences]
    lengths = np.asarray([array.shape[0] for array in arrays], dtype=np.int32)
    maxlen = np.max(lengths)
    arrays = [np.pad(array, [(0, maxlen - array.shape[0]), (0, 0)], 'constant', constant_values=0) for array in arrays]
    return np.asarray(arrays), lengths
    

In [3]:
small_cnf, small_lengths = pad_and_concat(
    [[[1, -2], [2, 1]],
     [[-2, -1], [1, -2]],
     [[-1, -1], [-2, -2], [-1, -2]],
      [[1, -1]],
      [[1, -1], [1, 2]],
      [[-2, 2]]
     ])
small_sol = np.asarray([1, 1, 1, 0, 0, 0])

In [4]:
print(small_cnf)
print(small_lengths)

[[[ 1 -2]
  [ 2  1]
  [ 0  0]]

 [[-2 -1]
  [ 1 -2]
  [ 0  0]]

 [[-1 -1]
  [-2 -2]
  [-1 -2]]

 [[ 1 -1]
  [ 0  0]
  [ 0  0]]

 [[ 1 -1]
  [ 1  2]
  [ 0  0]]

 [[-2  2]
  [ 0  0]
  [ 0  0]]]
[2 2 3 1 2 1]


In [5]:
VARIABLE_NUM = 3
EMBEDDING_SIZE = 8
CLAUSE_SIZE = 2
LSTM_STATE_SIZE = 8
batch_size = 6

In [6]:
def assert_shape(matrix, shape: list):
    act_shape = matrix.get_shape().as_list()
    assert act_shape == shape, "got shape {}, expected {}".format(act_shape, shape)

In [7]:
class Graph:
    def __init__(self):
        self.inputs = tf.placeholder(tf.int32, shape=(batch_size, None, CLAUSE_SIZE), name='inputs')
        self.lengths = tf.placeholder(tf.int32, shape=(batch_size,), name='lengths')
        self.labels = tf.placeholder(tf.float32, shape=(batch_size,), name='labels')
        
        vars_ = tf.abs(self.inputs)
        signs = tf.cast(tf.sign(self.inputs), tf.float32)  # shape: [batch_size, None, CLAUSE_SIZE]

        embeddings = tf.Variable(tf.random_uniform([VARIABLE_NUM + 1, EMBEDDING_SIZE], -1., 1), name='embeddings')

        var_embeddings = tf.nn.embedding_lookup(embeddings, vars_)
        # var_embeddings shape: [None, None, CLAUSE_SIZE, EMBEDDING_SIZE]
        
        clause_preembeddings = tf.concat(
            [tf.reshape(var_embeddings, [batch_size, -1, CLAUSE_SIZE * EMBEDDING_SIZE]), 
             signs],
            axis=2)
        
        PREEMBEDDING_SIZE = EMBEDDING_SIZE * CLAUSE_SIZE + CLAUSE_SIZE
        assert_shape(clause_preembeddings, 
                     [batch_size, None, PREEMBEDDING_SIZE])
        
        clause_w = tf.Variable(tf.random_normal(
            [PREEMBEDDING_SIZE, EMBEDDING_SIZE]), name='clause_w')
        clause_b = tf.Variable(tf.random_normal([EMBEDDING_SIZE]), name='clause_b')
        clause_embeddings = tf.reshape(tf.sigmoid(
            tf.reshape(clause_preembeddings, [-1, PREEMBEDDING_SIZE]) @ clause_w + clause_b), 
                                       [batch_size, -1, EMBEDDING_SIZE])
        # shape: [None, None, EMBEDDING_SIZE]
        
        lstm = tf.contrib.rnn.BasicLSTMCell(LSTM_STATE_SIZE)
        hidden_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        current_state = tf.zeros([batch_size, LSTM_STATE_SIZE])
        state = hidden_state, current_state
        
        _, lstm_final_state = tf.nn.dynamic_rnn(lstm, clause_embeddings, dtype=tf.float32, 
                                               sequence_length=self.lengths
                                               )
        formula_embedding = lstm_final_state.h
            
        assert_shape(formula_embedding, [batch_size, LSTM_STATE_SIZE])
            
        softmax_w = tf.Variable(tf.random_normal([LSTM_STATE_SIZE, 1]), name='softmax_w')
        softmax_b = tf.Variable(tf.random_normal([1]), name='softmax_b')
        
        self.logits = tf.squeeze(formula_embedding @ softmax_w, axis=1) + softmax_b
        self.loss = tf.losses.sigmoid_cross_entropy(self.labels, self.logits) 
        self.probabilities = tf.sigmoid(self.logits)

tf.reset_default_graph()
model = Graph()

In [8]:
np.set_printoptions(precision=2, suppress=True)

In [9]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    probs = sess.run([model.probabilities], feed_dict={
        model.inputs: small_cnf,
        model.lengths: small_lengths
    })
    
print(probs)

[array([0.44, 0.46, 0.5 , 0.4 , 0.42, 0.4 ], dtype=float32)]


In [10]:
with tf.Session() as sess:
    train_op = tf.train.AdamOptimizer(learning_rate=0.1).minimize(model.loss)
    sess.run(tf.global_variables_initializer())
    for _ in range(100):
        sess.run(train_op, feed_dict={
            model.inputs: small_cnf,
            model.labels: small_sol,
            model.lengths: small_lengths
        })
    probs = sess.run(model.probabilities, feed_dict={
        model.inputs: small_cnf,
        model.lengths: small_lengths
    })
    
probs

array([1., 1., 1., 0., 0., 0.], dtype=float32)

In [11]:
import cnf

In [12]:
def sat_array():
    for k in range(1, 6):
        for var_num in range(2, 5):
            for clause_num in range(3, 10):
                sat = {True: 0, False: 0}
                for _ in range(100):
                    sat[cnf.get_random_kcnf(k, var_num, clause_num).satisfiable()] += 1
                print(k, var_num, clause_num, sat)
# sat_array()

In [13]:
cnfs = cnf.get_random_kcnfs(10000, 2, 3, 10)

In [14]:
labels = [cnf.satisfiable() for cnf in cnfs]

In [15]:
print("sat:", sum(1 for label in labels if label == True), "unsat:", sum(1 for label in labels if label == False))

sat: 7510 unsat: 2490


In [16]:
def chunks(lists, chunk_size):
    return [[it[i:i + chunk_size] for it in lists] for i in range(0, len(lists[0]), chunk_size)]

In [17]:
chunks([list(range(10)), list(range(10, 20))], 2)

[[[0, 1], [10, 11]],
 [[2, 3], [12, 13]],
 [[4, 5], [14, 15]],
 [[6, 7], [16, 17]],
 [[8, 9], [18, 19]]]

In [18]:
with tf.Session() as sess:
    train_op = tf.train.AdamOptimizer(learning_rate=0.01).minimize(model.loss)
    sess.run(tf.global_variables_initializer())
    
    for epoch_num in range(400):
        print("Epoch", epoch_num)
        losses = []
        accs = []
        for (batch_cnfs, batch_labels) in chunks((cnfs, labels), batch_size):
            if len(batch_cnfs) < batch_size:
                print("skipping incomplete batch")
                continue
            inputs, lengths = pad_and_concat([cnf.clauses for cnf in batch_cnfs])
            _, loss, probs = sess.run([train_op, model.loss, model.probabilities], feed_dict={
                model.inputs: inputs,
                model.labels: batch_labels,
                model.lengths: lengths
            })
            losses.append(loss)
            accs.append(1 - np.mean(np.abs(np.around(probs) - np.asarray(batch_labels))))
            
        print("loss:", np.mean(np.asarray(losses)), "acc:", np.mean(accs))

    print("first batch:")
    first_inputs, first_lengths = pad_and_concat([cnf.clauses for cnf in cnfs[:batch_size]])
    probs = sess.run(model.probabilities, feed_dict={
        model.inputs: first_inputs,
        model.lengths: first_lengths
    })
    print("actual:", probs)
    print("expected", labels[:batch_size])
        
    print("new data:")
    probs = sess.run(model.probabilities, feed_dict={
        model.inputs: small_cnf,
        model.lengths: small_lengths
    })
    print("actual", probs)
    print("expected", small_sol)


Epoch 0
skipping incomplete batch
loss: 0.4299104 acc: 0.7691076380901691
Epoch 1
skipping incomplete batch
loss: 0.40379146 acc: 0.7912164817587668
Epoch 2
skipping incomplete batch
loss: 0.34892902 acc: 0.8250300076578846
Epoch 3
skipping incomplete batch
loss: 0.3093779 acc: 0.8455382114251693
Epoch 4
skipping incomplete batch
loss: 0.2962058 acc: 0.8506402522772729
Epoch 5
skipping incomplete batch
loss: 0.29038393 acc: 0.8518407325140711
Epoch 6
skipping incomplete batch
loss: 0.28698534 acc: 0.8532412927441236
Epoch 7
skipping incomplete batch
loss: 0.28511658 acc: 0.8553421331339237
Epoch 8
skipping incomplete batch
loss: 0.2826699 acc: 0.8561424533126591
Epoch 9
skipping incomplete batch
loss: 0.2844786 acc: 0.8551420530870039
Epoch 10
skipping incomplete batch
loss: 0.2780867 acc: 0.8602440940285502
Epoch 11
skipping incomplete batch
loss: 0.27562436 acc: 0.8629451745054444
Epoch 12
skipping incomplete batch
loss: 0.27344155 acc: 0.861944774279789
Epoch 13
skipping incomplete 